# Create Vector DB of corpus embeddings

In [1]:
import pandas as pd
import constants
import chromadb
from sentence_transformers import util, SentenceTransformer, CrossEncoder
import torch
from tqdm.autonotebook import tqdm, trange

/Users/shubham/Desktop/shubham/upliance/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
corpus_df = pd.read_parquet(constants.recipe_file, columns=['id', 'document'])#.sample(50)
corpus_df.head()

,id,document
0,54087d1b-6dab-403b-93eb-c33162f10aae,Recipe name is Zucchini Curry.\nZucchini curry...
1,b7f22d40-8a95-4877-8d50-0f493c61738a,Recipe name is Gujarati Kadhi.\nGujarati kadhi...
2,a15df9da-6efd-4bf1-ba6e-0f5fa0fc6a4e,Recipe name is Sabudana Khichdi.\nSabudana Khi...
3,f5e954c2-821a-4541-a3b0-d126baa02ac5,Recipe name is Beetroot Palya.\nThis recipe is...
4,f7b53d60-5c24-4512-bba5-90af1f27f8ce,Recipe name is Jain Red Sauce Pasta .\nRed sau...


In [3]:
# Initialising vector db
chroma_client = chromadb.PersistentClient(path=constants.vector_db_path)

# switch `create_collection` to `get_or_create_collection` to avoid creating a new collection every time
collection = chroma_client.get_or_create_collection(
    name=constants.recipe_collection, 
    metadata={"hnsw:space": "cosine"}   # use cosine similarity
    )


In [4]:
# load bi-encoder model 
model = SentenceTransformer(constants.retrieval_model_id, 
                            cache_folder=constants.retrieval_model_cache_path, 
                            device=constants.device)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/319 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# create corpus embeddings
corpus_embeddings = model.encode(corpus_df['document'].to_list(), convert_to_tensor=True)

In [6]:
# add embeddings to vector db
collection.upsert(
    embeddings=corpus_embeddings.tolist(),
    ids=corpus_df['id'].to_list()
)
